In [1]:
import os
import textwrap

import chromadb
import langchain
import openai
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import WebBaseLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.indexes import VectorstoreIndexCreator
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.vectorstores import Chroma
from langchain.document_loaders import PyPDFLoader
from langchain.vectorstores import FAISS


from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA, ConversationalRetrievalChain
import os

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [3]:
def print_response(response: str):
    print("\n".join(textwrap.wrap(response, width=100)))

In [5]:
os.environ["OPENAI_API_KEY"] = ""
openai.api_key = "

In [7]:
def chat_completion(prompt, model="gpt-3.5-turbo", temperature=0):
    res = openai.ChatCompletion.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        temperature=temperature,
    )
    print(res["choices"][0]["message"]["content"])

In [9]:
chat_completion("Hi")

Hello! How can I assist you today?


In [11]:
import PyPDF2

def append_pdf(input_pdf, output_pdf):
    """
    Append pages of input_pdf to output_pdf.

    Parameters:
        input_pdf (str): Path to the input PDF file.
        output_pdf (str): Path to the output PDF file.
    """
    # Open the input PDF file in read-binary mode
    with open(input_pdf, 'rb') as input_pdf_file:
        # Create PdfFileReader object for the input PDF
        input_pdf_reader = PyPDF2.PdfReader(input_pdf_file)

        # Add all pages of input PDF to the output PDF writer
        for page_num in range(len(input_pdf_reader.pages)):
            page = input_pdf_reader.pages[page_num]
            output_pdf_writer.add_page(page)

# Paths to the input PDF files
input_pdf2 = 'SOFI-2023.pdf'
input_pdf1 = 'SOFI-2024.pdf'
output_pdf = 'CMPE-combined-HackathonContent.pdf'

# Create a PdfFileWriter object for the output PDF
output_pdf_writer = PyPDF2.PdfWriter()

# Append content of input_pdf1 to output_pdf
append_pdf(input_pdf1, output_pdf_writer)

# Append content of input_pdf2 to output_pdf
append_pdf(input_pdf2, output_pdf_writer)

# Write the output PDF to the output file
with open(output_pdf, 'wb') as output_pdf_file:
    output_pdf_writer.write(output_pdf_file)

In [13]:
import PyPDF2
#import pdf2image
from PIL import Image
#import pytesseract
import os
import openai


#openai.api_key = "{Your OpenAI API Key}"
# Open the PDF file
pdf_file_path = "CMPE-combined-HackathonContent.pdf"

pdf_file = open(pdf_file_path, 'rb')

# Create a PDF object
pdf_reader = PyPDF2.PdfReader(pdf_file)

# Initialize a variable to store the text from the PDF
detected_text = ''
print(len(pdf_reader.pages))
# Loop through all the pages and extract text
for page_num in range(len(pdf_reader.pages)):
  print(page_num)
  page = pdf_reader.pages[page_num]
  print(page.extract_text())
  detected_text += page.extract_text() + '\n\n'

# Close the PDF file
pdf_file.close()

# Now, 'pdf_text' contains all the text from the PDF



602
0
FOOD SECURITY  AND NUTRITION  IN THE WORLD
THE STATE OF 
FINANCING TO END HUNGER,  
FOOD INSECURITY AND MALNUTRITION  
IN ALL ITS FORMS
2024
1
INDIA.  Winnowing wheat: scaling up financing flows will accelerate the vital transformation of agrifood systems.COVER PHOTOGRAPH  © iStock.com/pixelfusion3dThis flagship publication is part of The State of the World series of the Food and Agriculture Organization  
of the United Nations.
Required citation:
FAO, IFAD, UNICEF, WFP and WHO. 2024. The State of Food Security and Nutrition in the World 2024 – Financing to end 
hunger, food insecurity and malnutrition in all its forms . Rome.
https://doi.org/10.4060/cd1254en
The designations employed and the presentation of material in this information product do not imply the expression of any 
opinion whatsoever on the part of the Food and Agriculture Organization of the United Nations (FAO), the International Fund 
for Agricultural Development (IFAD), the United Nations Children’s Fund (UNICE

In [14]:
# Re-adjust chunking
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=100)
texts = text_splitter.create_documents([detected_text])

# Re-create the FAISS index with adjusted parameters
directory = 'index_store'
vector_index = FAISS.from_documents(texts, OpenAIEmbeddings())
vector_index.save_local(directory)

# Load and increase retrieval scope
vector_index = FAISS.load_local('index_store', OpenAIEmbeddings(), allow_dangerous_deserialization=True)
retriever = vector_index.as_retriever(search_type="similarity", search_kwargs={"k": 10})

# Set up the refined chain type
qa_interface = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(temperature=0.3), 
    chain_type="refine", 
    retriever=retriever, 
    return_source_documents=True
)

# Test query with refined settings
response = qa_interface("Compare food insecurity reasons between 2023 and 2024")
print(response["result"])



/var/folders/10/b_3jtvwj7hxfslxknmygr6hc0000gn/T/ipykernel_78465/1730211897.py:7: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  vector_index = FAISS.from_documents(texts, OpenAIEmbeddings())
/var/folders/10/b_3jtvwj7hxfslxknmygr6hc0000gn/T/ipykernel_78465/1730211897.py:16: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm=ChatOpenAI(temperature=0.3),
/var/folders/10/b_3jtvwj7hxfsl

The additional context provided highlights the specific regions in Africa where food insecurity worsened in 2022. This information can be used to further refine the comparison of food insecurity reasons between 2023 and 2024. In 2023, global trends in hunger were influenced by various factors, including the income effect from economic recovery and employment growth in low-income and lower-middle-income countries. These factors likely played a role in improving food security for some populations, but challenges such as high inflation rates could have offset these gains, particularly for the most vulnerable households.

In 2024, it will be important to consider how these global trends in food insecurity intersect with the regional dynamics observed in Northern Africa, Middle Africa, Southern Africa, and Western Africa in 2022. Understanding the specific challenges faced by these regions, such as the increase in severe food insecurity and the additional millions of people affected, can pr

In [27]:
import gradio as gr

def chat(chat_history, user_input):
  bot_response = qa_interface(user_input)['result']
  #print(bot_response)
  response = ""
  for letter in ''.join(bot_response): #[bot_response[i:i+1] for i in range(0, len(bot_response), 1)]:
      response += letter + ""
      yield chat_history + [(user_input, response)]

with gr.Blocks() as demo:
    gr.Markdown('# Economic and Social Sustainability of Marginalized and Highly Vulnerable communities ')
    with gr.Tab("Knowledge Bot"):
#          inputbox = gr.Textbox("Input your text to build a Q&A Bot here.....")
          chatbot = gr.Chatbot()
          message = gr.Textbox ("")
          message.submit(chat, [chatbot, message], chatbot)

demo.queue().launch(debug = True)

/opt/anaconda3/lib/python3.12/site-packages/gradio/components/chatbot.py:223: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Keyboard interruption in main thread... closing server.


In [29]:
import gradio as gr

def chat(chat_history, user_input):
    bot_response = qa_interface(user_input)['result']
    response = ""
    for letter in bot_response:
        response += letter
        yield chat_history + [(user_input, response)]

with gr.Blocks(css="""
    .gradio-container {
        background-color: #ffffff;  /* White background */
        color: #333333;  /* Dark gray text for general content */
    }
    .chatbot .message.user {
        background-color: #d1e7dd;
        border-radius: 10px;
        padding: 10px;
        color: #333333;
    }
    .chatbot .message.bot {
        background-color: #e2e3e5;
        border-radius: 10px;
        padding: 10px;
        color: #333333;
    }
    .textbox input {
        border-radius: 5px;
        border: 1px solid #ced4da;
        padding: 10px;
        color: #333333;
        background-color: #ffffff;  /* White input box */
    }
    .gradio-container h1 {
        color: #000000;  /* Black header text */
        font-size: 24px;
        font-weight: bold;
        text-align: center;
    }
""") as demo:
    gr.Markdown("<h1>Economic and Social Sustainability of Marginalized and Highly Vulnerable Communities</h1>")
    
    with gr.Tab("Knowledge Bot"):
        chatbot = gr.Chatbot(label="Knowledge Bot", elem_classes=["chatbot"], show_label=False)
        message = gr.Textbox(placeholder="Type your question here...", show_label=False)
        message.submit(chat, [chatbot, message], chatbot)

demo.queue().launch(debug=True)


/opt/anaconda3/lib/python3.12/site-packages/gradio/components/chatbot.py:223: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Keyboard interruption in main thread... closing server.


In [34]:
!pip install chromadb
!pip install PyPDF2
!pip install langchain
! pip install openai==0.27.8
! pip install cohere
! pip install kaleido
!pip install chromadb
!pip install PyPDF2
!pip install langchain
#!pip install openai
#!pip install llama-index
!pip install pypdf
!pip install gradio
!pip install langchain_community
! pip install pycryptodome==3.15.0
! pip install tiktoken
!pip install faiss-cpu
!pip install gradio

^C
Traceback (most recent call last):
  File "/opt/anaconda3/bin/pip", line 8, in <module>
    sys.exit(main())
             ^^^^^^
  File "/opt/anaconda3/lib/python3.12/site-packages/pip/_internal/cli/main.py", line 78, in main
    command = create_command(cmd_name, isolated=("--isolated" in cmd_args))
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/lib/python3.12/site-packages/pip/_internal/commands/__init__.py", line 114, in create_command
    module = importlib.import_module(module_path)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/lib/python3.12/importlib/__init__.py", line 90, in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<frozen importlib._bootstrap>", line 1387, in _gcd_import
  File "<frozen importlib._bootstrap>", line 1360, in _find_and_load
  File "<frozen importlib._bootstrap>", line 1331, 